In [1]:
#1 Loading the Data
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
if os.path.isdir('Graphs') == False:
    os.mkdir('Graphs')
burglar_df = pd.read_csv('Burglars.csv')

In [2]:
#2 Grouping by LSOA
groups = burglar_df.groupby('LSOA name')
grouped_df = {name: df for name, df in groups}

In [3]:
#3 Define month-grabbing function:
def get_per_month_count(unf_df, fil_df):
    months = unf_df['Month'].unique()
    per_month = {}
    for i in months:
        per_month[i] = len(fil_df[fil_df['Month'] == i])
    return per_month

In [4]:
#4 Generating the per-month-year bar charts and dictionaries:
LSOA_per_month = {}
for i in grouped_df.keys():
    df = grouped_df[i]
    groups = df.groupby('Month')
    month_df = get_per_month_count(burglar_df, df)
    LSOA_per_month[i] = month_df
    months = list(month_df.keys())
    step = max(1, len(months) // 15)


In [5]:
#5 Automatic Recency problem analysis: Define Capturing of Points and Gaps
print(LSOA_per_month.keys())
point_gap = {}
for i in LSOA_per_month.keys():
    df = LSOA_per_month[i]
    gap = []
    point = []
    neither = []
    for month in df.keys():
        if df[month] == 0:
            gap.append(month)
        elif df[month] == 1:
            point.append(month)
        else:
            neither.append(month)
    point_gap[i] = (point, gap, neither)
    print(f'LSOA {i}')


dict_keys(['Barking and Dagenham 001A', 'Barking and Dagenham 001B', 'Barking and Dagenham 001C', 'Barking and Dagenham 001D', 'Barking and Dagenham 002A', 'Barking and Dagenham 002B', 'Barking and Dagenham 002C', 'Barking and Dagenham 002D', 'Barking and Dagenham 002E', 'Barking and Dagenham 002F', 'Barking and Dagenham 003A', 'Barking and Dagenham 003B', 'Barking and Dagenham 003C', 'Barking and Dagenham 003D', 'Barking and Dagenham 004A', 'Barking and Dagenham 004B', 'Barking and Dagenham 004C', 'Barking and Dagenham 004D', 'Barking and Dagenham 004E', 'Barking and Dagenham 004F', 'Barking and Dagenham 004G', 'Barking and Dagenham 006A', 'Barking and Dagenham 006B', 'Barking and Dagenham 006C', 'Barking and Dagenham 006D', 'Barking and Dagenham 006E', 'Barking and Dagenham 007A', 'Barking and Dagenham 007B', 'Barking and Dagenham 007C', 'Barking and Dagenham 007D', 'Barking and Dagenham 007E', 'Barking and Dagenham 007F', 'Barking and Dagenham 007G', 'Barking and Dagenham 008A', 'Ba

LSOA Havering 025D
LSOA Havering 025E
LSOA Havering 026A
LSOA Havering 026B
LSOA Havering 026C
LSOA Havering 026D
LSOA Havering 026E
LSOA Havering 027A
LSOA Havering 027B
LSOA Havering 027C
LSOA Havering 027D
LSOA Havering 027E
LSOA Havering 028A
LSOA Havering 028B
LSOA Havering 028C
LSOA Havering 028D
LSOA Havering 028E
LSOA Havering 029A
LSOA Havering 029B
LSOA Havering 029C
LSOA Havering 029D
LSOA Havering 030A
LSOA Havering 030B
LSOA Havering 030C
LSOA Havering 030D
LSOA Hillingdon 001A
LSOA Hillingdon 001B
LSOA Hillingdon 001C
LSOA Hillingdon 001D
LSOA Hillingdon 001E
LSOA Hillingdon 002A
LSOA Hillingdon 002B
LSOA Hillingdon 002C
LSOA Hillingdon 002D
LSOA Hillingdon 002E
LSOA Hillingdon 002F
LSOA Hillingdon 003A
LSOA Hillingdon 003B
LSOA Hillingdon 003C
LSOA Hillingdon 003D
LSOA Hillingdon 003E
LSOA Hillingdon 004A
LSOA Hillingdon 004B
LSOA Hillingdon 004C
LSOA Hillingdon 004D
LSOA Hillingdon 004E
LSOA Hillingdon 005A
LSOA Hillingdon 005B
LSOA Hillingdon 005C
LSOA Hillingdon 005D


In [6]:
durations = {}
df = burglar_df['Month'].unique()
len_ = len(point_gap.keys())
count = 0
prevp = 0
for i in point_gap.keys():
    print(i)
    distance = []
    point = point_gap[i][0]
    gap = point_gap[i][1]
    neither = point_gap[i][2]
    for month in df:
        if month in point:
            to_gap = 0
            dt_month = datetime.strptime(month, '%Y-%m')
            gap_c = [datetime.strptime(date, '%Y-%m') for date in gap]
            for j in range(12):
                current_date = dt_month + timedelta(days=30*j)  # adding approximately a month each time

                # Check if the current date is in the gap list
                if current_date in gap_c:
                    current_string = current_date.strftime('%Y-%m')
                    print(f'Point at {month}, Gap at {current_string}.')
                    distance.append(to_gap)
                    break
                else:
                    to_gap += 1
    durations[i] = distance
    


Barking and Dagenham 001A
Point at 2016-09, Gap at 2016-10.
Point at 2021-04, Gap at 2021-05.
Point at 2022-02, Gap at 2022-06.
Barking and Dagenham 001B
Point at 2011-04, Gap at 2011-05.
Point at 2012-02, Gap at 2012-04.
Point at 2013-01, Gap at 2013-05.
Point at 2013-02, Gap at 2013-06.
Point at 2014-01, Gap at 2014-05.
Point at 2016-04, Gap at 2016-05.
Point at 2018-01, Gap at 2018-04.
Point at 2018-06, Gap at 2018-07.
Point at 2019-11, Gap at 2019-12.
Point at 2020-04, Gap at 2020-05.
Point at 2022-06, Gap at 2022-07.
Point at 2022-12, Gap at 2023-03.
Point at 2024-01, Gap at 2024-03.
Point at 2024-02, Gap at 2024-04.
Point at 2024-06, Gap at 2024-07.
Point at 2024-11, Gap at 2024-12.
Barking and Dagenham 001C
Point at 2012-04, Gap at 2012-05.
Point at 2015-01, Gap at 2015-04.
Point at 2017-06, Gap at 2017-07.
Point at 2018-11, Gap at 2019-03.
Point at 2019-11, Gap at 2019-12.
Point at 2020-04, Gap at 2020-05.
Point at 2022-01, Gap at 2022-04.
Point at 2022-11, Gap at 2022-12.
Poin

Point at 2011-06, Gap at 2011-07.
Point at 2011-09, Gap at 2011-10.
Point at 2012-02, Gap at 2012-05.
Point at 2012-04, Gap at 2012-05.
Point at 2014-01, Gap at 2014-04.
Point at 2017-02, Gap at 2017-06.
Point at 2017-04, Gap at 2017-05.
Point at 2017-11, Gap at 2017-12.
Point at 2019-06, Gap at 2019-07.
Point at 2020-06, Gap at 2020-07.
Point at 2020-12, Gap at 2021-03.
Point at 2021-09, Gap at 2021-10.
Point at 2022-02, Gap at 2022-07.
Point at 2022-06, Gap at 2022-07.
Point at 2023-01, Gap at 2023-04.
Point at 2023-02, Gap at 2023-07.
Point at 2023-06, Gap at 2023-07.
Point at 2024-01, Gap at 2024-03.
Point at 2024-11, Gap at 2024-12.
Barking and Dagenham 006B
Point at 2015-11, Gap at 2015-12.
Point at 2016-11, Gap at 2017-03.
Point at 2016-12, Gap at 2017-03.
Point at 2018-02, Gap at 2018-06.
Point at 2019-01, Gap at 2019-04.
Point at 2020-09, Gap at 2020-10.
Point at 2020-12, Gap at 2021-03.
Point at 2021-02, Gap at 2021-07.
Point at 2021-12, Gap at 2022-03.
Point at 2022-01, Gap 

Point at 2024-09, Gap at 2024-10.
Barking and Dagenham 009A
Point at 2011-11, Gap at 2011-12.
Point at 2013-04, Gap at 2013-05.
Point at 2014-06, Gap at 2014-07.
Point at 2015-01, Gap at 2015-05.
Point at 2015-02, Gap at 2015-06.
Point at 2015-04, Gap at 2015-05.
Point at 2015-09, Gap at 2015-10.
Point at 2016-04, Gap at 2016-05.
Point at 2018-12, Gap at 2019-03.
Point at 2019-01, Gap at 2019-04.
Point at 2020-02, Gap at 2020-04.
Point at 2020-11, Gap at 2021-03.
Point at 2020-12, Gap at 2021-03.
Point at 2023-09, Gap at 2023-10.
Point at 2024-06, Gap at 2024-07.
Barking and Dagenham 009B
Point at 2012-12, Gap at 2013-03.
Point at 2013-01, Gap at 2013-04.
Point at 2013-06, Gap at 2013-07.
Point at 2015-06, Gap at 2015-07.
Point at 2016-09, Gap at 2016-10.
Point at 2018-02, Gap at 2018-07.
Point at 2018-06, Gap at 2018-07.
Point at 2018-09, Gap at 2018-10.
Point at 2019-09, Gap at 2019-10.
Point at 2020-04, Gap at 2020-05.
Point at 2021-01, Gap at 2021-05.
Point at 2021-09, Gap at 2021-

Point at 2015-01, Gap at 2015-04.
Point at 2016-01, Gap at 2016-03.
Point at 2016-09, Gap at 2016-10.
Point at 2017-01, Gap at 2017-04.
Point at 2018-02, Gap at 2018-06.
Point at 2019-11, Gap at 2019-12.
Point at 2020-04, Gap at 2020-05.
Point at 2020-09, Gap at 2020-10.
Point at 2021-04, Gap at 2021-05.
Point at 2021-11, Gap at 2021-12.
Point at 2024-01, Gap at 2024-03.
Point at 2024-04, Gap at 2024-05.
Barking and Dagenham 013F
Point at 2011-06, Gap at 2011-07.
Point at 2013-09, Gap at 2013-10.
Point at 2014-02, Gap at 2014-06.
Point at 2014-04, Gap at 2014-05.
Point at 2015-09, Gap at 2015-10.
Point at 2016-12, Gap at 2017-03.
Point at 2017-11, Gap at 2017-12.
Point at 2020-11, Gap at 2020-12.
Point at 2021-02, Gap at 2021-06.
Point at 2022-06, Gap at 2022-07.
Point at 2023-01, Gap at 2023-04.
Point at 2024-02, Gap at 2024-04.
Barking and Dagenham 014A
Point at 2014-01, Gap at 2014-05.
Point at 2014-04, Gap at 2014-05.
Point at 2016-09, Gap at 2016-10.
Point at 2017-01, Gap at 2017-

Point at 2021-11, Gap at 2021-12.
Point at 2022-01, Gap at 2022-05.
Point at 2022-04, Gap at 2022-05.
Point at 2023-06, Gap at 2023-07.
Point at 2024-02, Gap at 2024-05.
Point at 2024-04, Gap at 2024-05.
Barking and Dagenham 018A
Point at 2016-01, Gap at 2016-03.
Point at 2017-01, Gap at 2017-05.
Point at 2017-04, Gap at 2017-05.
Point at 2018-01, Gap at 2018-04.
Point at 2018-06, Gap at 2018-07.
Point at 2019-02, Gap at 2019-06.
Point at 2020-02, Gap at 2020-05.
Point at 2020-04, Gap at 2020-05.
Point at 2020-11, Gap at 2021-03.
Point at 2020-12, Gap at 2021-03.
Point at 2021-11, Gap at 2022-03.
Point at 2023-01, Gap at 2023-04.
Point at 2024-01, Gap at 2024-03.
Barking and Dagenham 018B
Point at 2011-01, Gap at 2011-04.
Point at 2011-02, Gap at 2011-06.
Point at 2011-11, Gap at 2011-12.
Point at 2014-09, Gap at 2014-10.
Point at 2015-04, Gap at 2015-05.
Point at 2016-01, Gap at 2016-03.
Point at 2016-09, Gap at 2016-10.
Point at 2016-12, Gap at 2017-03.
Point at 2019-09, Gap at 2019-

...

Point at 2024-11, Gap at 2024-12.
Westminster 003D
Point at 2010-12, Gap at 2011-03.
Point at 2011-06, Gap at 2011-07.
Point at 2014-04, Gap at 2014-05.
Point at 2015-06, Gap at 2015-07.
Point at 2016-11, Gap at 2017-03.
Point at 2016-12, Gap at 2017-03.
Point at 2017-02, Gap at 2017-07.
Point at 2018-09, Gap at 2018-10.
Point at 2019-01, Gap at 2019-04.
Point at 2020-02, Gap at 2020-04.
Point at 2020-06, Gap at 2020-07.
Point at 2020-11, Gap at 2021-03.
Point at 2020-12, Gap at 2021-03.
Point at 2022-04, Gap at 2022-05.
Point at 2024-09, Gap at 2024-10.
Westminster 003E
Point at 2013-02, Gap at 2013-06.
Point at 2013-04, Gap at 2013-05.
Point at 2014-01, Gap at 2014-04.
Point at 2014-12, Gap at 2015-03.
Point at 2015-02, Gap at 2015-06.
Point at 2015-04, Gap at 2015-05.
Point at 2016-06, Gap at 2016-07.
Point at 2018-01, Gap at 2018-05.
Point at 2018-04, Gap at 2018-05.
Point at 2018-11, Gap at 2018-12.
Point at 2020-02, Gap at 2020-04.
Point at 2023-01, Gap at 2023-04.
Point at 2023-

Point at 2012-09, Gap at 2012-10.
Point at 2014-11, Gap at 2015-03.
Point at 2014-12, Gap at 2015-03.
Point at 2015-02, Gap at 2015-07.
Point at 2015-06, Gap at 2015-07.
Point at 2017-04, Gap at 2017-05.
Point at 2019-02, Gap at 2019-06.
Point at 2020-11, Gap at 2020-12.
Point at 2021-01, Gap at 2021-04.
Point at 2021-02, Gap at 2021-07.
Point at 2021-06, Gap at 2021-07.
Point at 2022-02, Gap at 2022-07.
Point at 2022-06, Gap at 2022-07.
Westminster 007F
Point at 2011-01, Gap at 2011-04.
Point at 2014-11, Gap at 2015-03.
Point at 2016-02, Gap at 2016-05.
Point at 2016-04, Gap at 2016-05.
Point at 2020-02, Gap at 2020-04.
Point at 2022-01, Gap at 2022-04.
Point at 2022-06, Gap at 2022-07.
Point at 2022-09, Gap at 2022-10.
Westminster 007G
Point at 2011-06, Gap at 2011-07.
Point at 2012-09, Gap at 2012-10.
Point at 2012-11, Gap at 2013-03.
Point at 2012-12, Gap at 2013-03.
Point at 2013-02, Gap at 2013-06.
Point at 2013-11, Gap at 2013-12.
Point at 2015-06, Gap at 2015-07.
Point at 2015-

Point at 2021-06, Gap at 2021-07.
Westminster 012D
Point at 2013-09, Gap at 2013-10.
Point at 2014-12, Gap at 2015-03.
Point at 2015-02, Gap at 2015-07.
Point at 2015-06, Gap at 2015-07.
Westminster 012E
Point at 2013-02, Gap at 2013-07.
Point at 2020-04, Gap at 2020-05.
Point at 2020-06, Gap at 2020-07.
Westminster 013A
Point at 2014-02, Gap at 2014-06.
Point at 2022-11, Gap at 2022-12.
Westminster 013B
Westminster 013D
Westminster 013E
Westminster 013F
Westminster 013G
Point at 2022-04, Gap at 2022-05.
Point at 2022-12, Gap at 2023-03.
Westminster 014A
Point at 2020-01, Gap at 2020-03.
Point at 2023-04, Gap at 2023-05.
Point at 2024-02, Gap at 2024-04.
Point at 2024-09, Gap at 2024-10.
Westminster 014B
Westminster 014D
Point at 2011-11, Gap at 2011-12.
Point at 2012-01, Gap at 2012-03.
Point at 2022-01, Gap at 2022-04.
Westminster 014E
Point at 2013-02, Gap at 2013-06.
Point at 2018-06, Gap at 2018-07.
Point at 2020-04, Gap at 2020-05.
Point at 2022-06, Gap at 2022-07.
Point at 2022-

Westminster 021A
Point at 2011-01, Gap at 2011-05.
Point at 2011-04, Gap at 2011-05.
Point at 2011-09, Gap at 2011-10.
Point at 2012-02, Gap at 2012-04.
Point at 2012-06, Gap at 2012-07.
Point at 2012-09, Gap at 2012-10.
Point at 2013-11, Gap at 2013-12.
Point at 2014-02, Gap at 2014-06.
Point at 2015-02, Gap at 2015-06.
Point at 2017-01, Gap at 2017-04.
Point at 2017-02, Gap at 2017-07.
Point at 2017-06, Gap at 2017-07.
Point at 2018-01, Gap at 2018-04.
Point at 2018-11, Gap at 2018-12.
Point at 2021-12, Gap at 2022-03.
Point at 2022-01, Gap at 2022-04.
Point at 2022-02, Gap at 2022-06.
Point at 2022-09, Gap at 2022-10.
Point at 2023-02, Gap at 2023-06.
Point at 2023-04, Gap at 2023-05.
Point at 2024-01, Gap at 2024-03.
Westminster 021B
Point at 2011-02, Gap at 2011-06.
Point at 2012-12, Gap at 2013-03.
Point at 2013-01, Gap at 2013-04.
Point at 2013-06, Gap at 2013-07.
Point at 2014-01, Gap at 2014-04.
Point at 2015-02, Gap at 2015-06.
Point at 2018-01, Gap at 2018-04.
Point at 2019-

Point at 2019-09, Gap at 2019-10.
Point at 2020-04, Gap at 2020-05.
Point at 2020-09, Gap at 2020-10.
Point at 2022-02, Gap at 2022-07.
Point at 2022-06, Gap at 2022-07.
Point at 2022-09, Gap at 2022-10.
Point at 2024-09, Gap at 2024-10.
Westminster 024C
Point at 2012-02, Gap at 2012-04.
Point at 2012-09, Gap at 2012-10.
Point at 2013-12, Gap at 2014-03.
Point at 2014-01, Gap at 2014-04.
Point at 2014-02, Gap at 2014-06.
Point at 2015-06, Gap at 2015-07.
Point at 2015-11, Gap at 2015-12.
Point at 2016-12, Gap at 2017-03.
Point at 2017-06, Gap at 2017-07.
Point at 2017-09, Gap at 2017-10.
Point at 2017-11, Gap at 2017-12.
Point at 2019-02, Gap at 2019-07.
Point at 2019-06, Gap at 2019-07.
Point at 2020-06, Gap at 2020-07.
Point at 2021-02, Gap at 2021-06.
Point at 2021-11, Gap at 2021-12.
Point at 2022-06, Gap at 2022-07.
Point at 2022-11, Gap at 2022-12.
Point at 2023-09, Gap at 2023-10.
Point at 2023-11, Gap at 2023-12.
Westminster 024D
Point at 2011-04, Gap at 2011-05.
Point at 2013-

In [7]:
#6 Calculate mean and stdev distance
mean_durations = {}
for i in durations.keys():
    mean_durations[i] = [(np.mean(durations[i]), np.std(durations[i])) if len(durations[i]) != 0 else 0][0]
print(mean_durations)

{'Barking and Dagenham 001A': (2.0, 1.4142135623730951), 'Barking and Dagenham 001B': (2.0, 1.1726039399558574), 'Barking and Dagenham 001C': (1.9230769230769231, 1.327898192433236), 'Barking and Dagenham 001D': (1.9473684210526316, 1.3562209186026448), 'Barking and Dagenham 002A': (1.5, 1.0), 'Barking and Dagenham 002B': (2.888888888888889, 1.5947444549341472), 'Barking and Dagenham 002C': (2.2857142857142856, 1.097306535409801), 'Barking and Dagenham 002D': (1.6666666666666667, 0.9428090415820634), 'Barking and Dagenham 002E': (2.4166666666666665, 1.4976833963009524), 'Barking and Dagenham 002F': (2.1666666666666665, 1.674979270186815), 'Barking and Dagenham 003A': (2.5, 1.284523257866513), 'Barking and Dagenham 003B': (2.0952380952380953, 1.4443572274383905), 'Barking and Dagenham 003C': (2.1052631578947367, 1.2934953397310007), 'Barking and Dagenham 003D': (2.533333333333333, 1.3597385369580757), 'Barking and Dagenham 004A': (2.1875, 1.3792547806696194), 'Barking and Dagenham 004B'